In [ ]:
# Libraries

import sys
import math
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [347]:
# Inital information

url = 'https://propertylink.estatesgazette.com/commercial-property-for-rent/bath/page-1?filters%5Bnot_property_states%5D%5B%5D=Under+Offer&filters%5Bto_let_price%5D%5Bcurrency%5D=gbp&filters%5Bto_let_price%5D%5Bfrom%5D=&filters%5Bto_let_price%5D%5Bsize_modifier%5D=sqft&filters%5Bto_let_price%5D%5Bto%5D=&search=Bath&sort_direction=desc&sort_field=created_at&view=grid'
# change page numbers for these!
html = requests.get(url)
soup = bs(html.text, 'html.parser')

body = soup.find('body')
header = body.find('div', class_ = 'results-header mt-5')
property_count = int(header.find('div', class_ = 'results-message').contents[0].strip())
pages_needed = math.ceil(property_count / 24) + 1 # what happens if only 3 are needed
print(f"There are {property_count} properties on the website currently.\nSo {pages_needed} pages are needed.")

There are 70 properties on the website currently.
So 4 pages are needed.


In [ ]:
# Function for getting links

def get_links(soup):
    links = []

    i = 1
    max_iterations = 30
    while len(links) < 24 and i <= max_iterations:
        elements = soup.select(f"body > div.container.mt-3 > div.results.row > div:nth-child({i}) > div.card.h-100.card--results > a")

        # if not elements:
        #     elements = soup.select(f"body > div.container.mt-3 > div.results.row > div:nth-child({i}) > div.card.h-100.card--featured > a")

        if not elements:
            i += 1
            continue

        for element in elements:
            href = element['href']
            link = 'https://propertylink.estatesgazette.com'+ href
            links.append(link)
        i += 1

    return links


all_links = get_links(soup)


In [349]:
# Looping through pages

base_url = 'https://propertylink.estatesgazette.com/commercial-property-for-rent/bath/page-{}?filters%5Bnot_property_states%5D%5B%5D=Under+Offer&filters%5Bto_let_price%5D%5Bcurrency%5D=gbp&filters%5Bto_let_price%5D%5Bfrom%5D=&filters%5Bto_let_price%5D%5Bsize_modifier%5D=sqft&filters%5Bto_let_price%5D%5Bto%5D=&search=Bath&sort_direction=desc&sort_field=created_at&view=grid'

all_links = []  # Store all links from all pages

for page_num in range(1, pages_needed+1):  # Loop over the page numbers (1 to 3 in this example)
    url = base_url.format(page_num)
    html = requests.get(url)
    soup = bs(html.text, 'html.parser')
    
    page_links = get_links(soup)
    all_links.extend(page_links)  # Add the links from this page to the all_links list

    print(f"Page {page_num} has {len(page_links)} links.")

property_links = list(set(all_links)) # removing duplicates
print(f"Total links collected: {len(property_links)}")


Page 1 has 23 links.
Page 2 has 23 links.
Page 3 has 23 links.
Page 4 has 1 links.
Total links collected: 70


In [355]:
# Single property test

single_link = property_links[2]
print(single_link)

html = requests.get(single_link)
soup = bs(html.text, 'html.parser')

# soup.select('body > div.details-section.details-agent-contacts > div > div > div > div.col-md-4.col-12 > div.agent-name')
agent_name = soup.find('div', class_='agent-name').find_all_next(string=True)[1].strip()


agency_name = soup.select_one('body > div.details-section.details-agent-contacts > div > div > div > div.col-md-4.col-12 > div.agent-logo > a > img')['alt']
agency_name = soup.find("meta", {"name": "gwa_advertiserName"})
agency_name = agency_name.get("content")

id_tag = soup.find("meta", {"name": "gwa_contentID"})
id_tag = id_tag.get("content")

# address = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div:nth-child(3)') # Full div
# address = address[0].get_text(strip=True) # gives address with below stuck to the front
# address = address.replace("ADDRESS", "").strip() #tidy

address = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > h1')
address = address[0].get_text(strip=True)
address = address.replace("sharebookmark", "").strip() #tidy

size = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div:nth-child(2)')
size = size[0].get_text(strip=True) # gives size with below stuck to the front
size = size.replace("SIZE", "").strip() #tidy

type = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div:nth-child(4)')
type = type[0].get_text(strip=True)
type = type.replace("TYPE", "").strip()

price = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div:nth-child(1)')
price = price[0].get_text(strip=True)
price = price.replace("TO RENT", "").strip()

date_posted = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div:nth-child(6)')
date_posted = date_posted[0].get_text(strip=True)
date_posted = date_posted.replace("POSTED ON", "").strip()

desc = soup.select_one('body > div:nth-of-type(4) > div:nth-of-type(2) > div > div:nth-of-type(5) > div > div:nth-of-type(1)')
desc = desc.get_text(strip=True)

latitude = soup.find('meta', itemprop='latitude')['content']
longitude = soup.find('meta', itemprop='longitude')['content']

print(agent_name)
print(agency_name)
print(id_tag)
print(address)
print(size)
print(type)
print(price)
print(date_posted)
print(desc,'\n')
print(latitude, ',', longitude)



https://propertylink.estatesgazette.com/property-details/6778007-a-b-amp-grd-c-d-crescent-office-park-clarks-way-bath-ba2-2af
James Preece
Colliers 
6778007
A/B & Grd C/D Crescent Office Park,  Clarks Way, Bath, BA2 2AF
2,400 - 7,200 Sq Ft
Office, Offices
ROA
1st July 2022
Modern air conditioned offices with views over Bath and the surrounding area.EPC RatingC (58)Description-Air conditioning-Suspended ceilings-LG7 lighting-Perimeter trunking-Kitchenette-Male, female and disabled WC’sLocationBath is a world heritage site in the county of Somerset, 97 miles west of London and 11 miles south-east of Bristol.Crescent Office Park is located adjacent to the A367, 2 miles from the city centre on the southern outskirts. The property benefits from being 1.6 miles from Bath Railway Station and just 1.2 miles from Oldfield Park Station.AccommodationA/B Ground Floor - 2,400 sq ft (223 sq m)A/B First Floor - 2,400 sq ft (223 sq m)C/D Ground Floor - 2,400 sq ft (223 sq m)Car Parking27 on site car p

In [351]:
# Collecting all property information

def assign_value(elements, keyword):
    for element in elements:
        text = element.get_text(strip=True)
        if keyword in text:
            return text.replace(keyword, "").strip()
    return None

property_data = []

for property in property_links:
    # print(property)
    html = requests.get(property)
    soup = bs(html.text, 'html.parser')

    agent_name = soup.find('div', class_='agent-name').find_all_next(string=True)[1].strip()
    
    agency_name = soup.find("meta", {"name": "gwa_advertiserName"})
    agency_name = agency_name.get("content")

    id_tag = soup.find("meta", {"name": "gwa_contentID"})
    id_tag = id_tag.get("content")

    desc = soup.select_one('body > div:nth-of-type(4) > div:nth-of-type(2) > div > div:nth-of-type(5) > div > div:nth-of-type(1)')
    desc = desc.get_text(strip=True)

    address = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > h1')
    address = address[0].get_text(strip=True)
    address = address.replace("sharebookmark", "").strip() #tidy

    elements = soup.select('body > div.wrapper > div.container-wrapper.bg-white > div > div:nth-child(5) > div.details-section-body.row > div')

    # Assign values based on keywords - thank you ChatGPT for this one...
    price = assign_value(elements, "TO RENT")
    size = assign_value(elements, "SIZE")
    # address = assign_value(elements, "ADDRESS")
    type = assign_value(elements, "TYPE")
    # tenure = assign_value(elements, "TENURE")
    date_posted = assign_value(elements, "POSTED ON")

    latitude = soup.find('meta', itemprop='latitude')['content']
    longitude = soup.find('meta', itemprop='longitude')['content']
    

    property_data.append({
        'Id': id_tag,
        'Address': address,
        'Price': price,
        'Size': size,
        'Property Type': type,
        'Agency Name': agency_name,
        'Agent Name': agent_name,
        'Description': desc,
        'Hyperlink': property,
        'Date Posted': date_posted,
        'Latitude': latitude,
        'Longitude': longitude
    })

    

df = pd.DataFrame(property_data)



df.head(2)

,Id,Address,Price,Size,Property Type,Agency Name,Agent Name,Description,Hyperlink,Date Posted,Latitude,Longitude
0,6404227,"Kings Court Parsonage Lane, Bath, Bath And North East Somerset, BA1",ROA,"5,687 - 17,007 Sq Ft","Office, Offices",Carter Jonas LLP,Philip Marshall,High Quality Open Plan Refurbished Offices To Let in Central Bath( Agency Pilot Software ref: 20...,https://propertylink.estatesgazette.com/property-details/6404227-kings-court-parsonage-lane-bath...,11th April 2024,51.381907,-2.3611175
1,6987096,"First And Second Floors , 1A Queen Square, Bath, Bath and North East Somerset, BA1 2HA","£21,000.00 Per Annum","594 - 1,396 Sq Ft","Office, Offices",CSquared (Previously Colston & Colston Chartered Surveyors),Maddie Pyles,TO LET First and Second Floor Offices - Only £15 per SqFt per annum.Access to the premises is ...,https://propertylink.estatesgazette.com/property-details/6987096-first-and-second-floors-1a-quee...,1st November 2024,51.3837898,-2.3628998
2,6778007,"A/B & Grd C/D Crescent Office Park, Clarks Way, Bath, BA2 2AF",ROA,"2,400 - 7,200 Sq Ft","Office, Offices",Colliers,James Preece,Modern air conditioned offices with views over Bath and the surrounding area.EPC RatingC (58)Des...,https://propertylink.estatesgazette.com/property-details/6778007-a-b-amp-grd-c-d-crescent-office...,1st July 2022,51.3622856,-2.3866287
3,6978357,"Third Floor , 35 Gay Street, Bath, Bath and North East Somerset, BA1 2NT","£4,000.00 Per Annum",139 Sq Ft,"Office, Offices",CSquared (Previously Colston & Colston Chartered Surveyors),Maddie Pyles,"35 Gay Street is a grand and attractive, Grade II listed period townhouse. The building is of ty...",https://propertylink.estatesgazette.com/property-details/6978357-third-floor-35-gay-street-bath-...,5th September 2024,51.3843283,-2.3632202
4,6942427,"13-14 George Street, Bath, Bath And North East Somerset, BA1 2EN","£69,950.00 Per Annum","2,873 Sq Ft","Retail - High Street, Restaurants/Cafes, Pubs/Bars/Clubs, Retail, Licensed & Leisure",CSquared (Previously Colston & Colston Chartered Surveyors),Nathan Clark,Description13-14 George Street offers flexible trading space over ground and basement levels and...,https://propertylink.estatesgazette.com/property-details/6942427-13-14-george-street-bath-bath-a...,11th June 2024,51.38476,-2.3631805


In [352]:
df.to_csv('EG_scraped_13.11.24.csv',index=False)

In [353]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 100)

print(f"""Hi {df.iloc[2]['Agent Name']},\n 
I see that {df.iloc[2]['Address']} has been taken off the wesbites. Please could you let me know the tenant, rent and lease length?\n
Thanks,\n
[Name]""")

print(f"\n\n\n {df.iloc[2]}")

Hi James Preece,
 
I see that A/B & Grd C/D Crescent Office Park,  Clarks Way, Bath, BA2 2AF has been taken off the wesbites. Could you let me know the tenant, rent and lease length?

Thanks,

[Name]



 Id                                                                                                           6778007
Address                                               A/B & Grd C/D Crescent Office Park,  Clarks Way, Bath, BA2 2AF
Price                                                                                                            ROA
Size                                                                                             2,400 - 7,200 Sq Ft
Property Type                                                                                        Office, Offices
Agency Name                                                                                                Colliers 
Agent Name                                                                                    

In [354]:
import folium
map = folium.Map(location=(51.38, -2.36), zoom_start=10)#location - the center of the map, zoom_start - the resolution
for index, row in df.iterrows():
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=row['Address']
    ).add_to(map)
map